# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [82]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


In [83]:
import gmaps.datasets

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [84]:
cities_df = pd.read_csv('..\output_data/cities.csv')

cities_df

,Unnamed: 0,city_ID,City,Country,lat,lon,temp,wind,cloudiness,humidity
0,0,3495137,Pedernales,DO,18.04,-71.74,20.50,0.71,51,82
1,1,2165087,Gold Coast,AU,-28.00,153.43,25.47,8.70,75,53
2,2,3874787,Punta Arenas,CL,-53.15,-70.92,8.95,10.30,71,66
3,3,1006984,East London,ZA,-33.02,27.91,18.56,9.00,22,78
4,4,3418910,Upernavik,GL,72.79,-56.15,-27.96,4.08,0,50
...,...,...,...,...,...,...,...,...,...,...
571,571,3372760,Praia da Vitória,PT,38.73,-27.07,16.00,6.91,40,82
572,572,3388847,São Félix do Xingu,BR,-6.64,-51.99,21.85,0.93,67,97
573,573,3980627,Valle de Allende,MX,26.93,-105.40,17.54,2.85,33,48
574,574,98182,Baghdad,IQ,33.34,44.40,14.00,2.10,27,82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [85]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)


In [86]:
figure_layout = {
    'width': '200px',
    'height': '200px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [89]:
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]]
humidity = cities_df["humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [90]:
#Testing with a max temperature lower than 40 degrees but higher than 20.
#Wind speed less than 5 mph.
#Zero cloudiness.

hotel_df = cities_df.loc[(cities_df['temp'] > 20) & (cities_df['temp'] < 40) & (cities_df['wind'] < 5) & (cities_df['cloudiness'] == 0)]
hotel_df = hotel_df.dropna()
hotel_df


,Unnamed: 0,city_ID,City,Country,lat,lon,temp,wind,cloudiness,humidity
35,35,292968,Abu Dhabi,AE,24.47,54.37,21.51,1.00,0,73
54,54,2063042,Port Hedland,AU,-20.32,118.57,35.00,2.60,0,50
104,104,3837213,San Juan,AR,-31.54,-68.54,22.54,1.11,0,52
132,132,1622318,Waingapu,ID,-9.66,120.26,29.24,1.96,0,66
152,152,1637001,Biak,ID,-0.91,122.88,28.25,3.89,0,74
185,185,3361934,Saldanha,ZA,-33.01,17.94,21.00,3.60,0,49
238,238,2167426,Emerald,AU,-23.53,148.17,28.39,4.60,0,42
321,321,4026082,Lázaro Cárdenas,MX,17.96,-102.20,23.91,1.04,0,85
322,322,1261077,Odlābāri,IN,26.83,88.62,21.96,1.23,0,25
333,333,3587498,Acajutla,SV,13.59,-89.83,23.89,0.89,0,87


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
#cities_df['Hotel Name'] = ''
hotel_df['Hotel Name'] = ''
hotel_df

,Unnamed: 0,city_ID,City,Country,lat,lon,temp,wind,cloudiness,humidity,Hotel Name
35,35,292968,Abu Dhabi,AE,24.47,54.37,21.51,1.00,0,73,
54,54,2063042,Port Hedland,AU,-20.32,118.57,35.00,2.60,0,50,
104,104,3837213,San Juan,AR,-31.54,-68.54,22.54,1.11,0,52,
132,132,1622318,Waingapu,ID,-9.66,120.26,29.24,1.96,0,66,
152,152,1637001,Biak,ID,-0.91,122.88,28.25,3.89,0,74,
185,185,3361934,Saldanha,ZA,-33.01,17.94,21.00,3.60,0,49,
238,238,2167426,Emerald,AU,-23.53,148.17,28.39,4.60,0,42,
321,321,4026082,Lázaro Cárdenas,MX,17.96,-102.20,23.91,1.04,0,85,
322,322,1261077,Odlābāri,IN,26.83,88.62,21.96,1.23,0,25,
333,333,3587498,Acajutla,SV,13.59,-89.83,23.89,0.89,0,87,


In [92]:
target_radius = 5000
target_type = "lodging"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
for i, row in hotel_df.iterrows():
    
    target_coordinates = str(row['lat']) + ',' + str(row['lon'])
    
    params = {
     "location": target_coordinates,
     "radius": target_radius,
     "types": target_type,
     "key": g_key
     }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    places_data = response.json()
    
    try:
        #store the hotel name      
        hotel_df.loc[i, 'Hotel Name'] = places_data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result... skip.")
    
        

Missing result... skip.


In [93]:
hotel_df

,Unnamed: 0,city_ID,City,Country,lat,lon,temp,wind,cloudiness,humidity,Hotel Name
35,35,292968,Abu Dhabi,AE,24.47,54.37,21.51,1.00,0,73,Beach Rotana
54,54,2063042,Port Hedland,AU,-20.32,118.57,35.00,2.60,0,50,The Esplanade Hotel
104,104,3837213,San Juan,AR,-31.54,-68.54,22.54,1.11,0,52,Hotel Provincial
132,132,1622318,Waingapu,ID,-9.66,120.26,29.24,1.96,0,66,Pondok Wisata Elim
152,152,1637001,Biak,ID,-0.91,122.88,28.25,3.89,0,74,Kost Ariz
185,185,3361934,Saldanha,ZA,-33.01,17.94,21.00,3.60,0,49,Blue Bay Lodge
238,238,2167426,Emerald,AU,-23.53,148.17,28.39,4.60,0,42,The Irish Village
321,321,4026082,Lázaro Cárdenas,MX,17.96,-102.20,23.91,1.04,0,85,Hotel Quinta Antigua
322,322,1261077,Odlābāri,IN,26.83,88.62,21.96,1.23,0,25,west damdim resort
333,333,3587498,Acajutla,SV,13.59,-89.83,23.89,0.89,0,87,Villavela Hotel Boutique


In [94]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [96]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig
# Display Map

Figure(layout=FigureLayout(height='420px'))